<a href="https://www.kaggle.com/code/faiqueali/tensorflow-movie-recommender-system?scriptVersionId=159359698" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<h1 style="font-family: Trebuchet MS; padding: 12px; font-size: 30px; color: #081c15; text-align: center; line-height: 1.25;">Recommender System<br><span style="color: #1b4332; font-size: 48px"><b>🎥Movies Recommendation📹</b></span><br><span style="color: #52b788; font-size: 20px">📀Using Deep Learning</span></h1>
<hr>

In [1]:
!pip install -q tensorflow-recommenders

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.9.0 requires pyarrow<6,>=1, but you have pyarrow 8.0.0 which is incompatible.
tensorflow-transform 1.9.0 requires pyarrow<6,>=1, but you have pyarrow 8.0.0 which is incompatible.
tensorflow-transform 1.9.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,<2.10,>=1.15.5, but you have tensorflow 2.11.0 which is incompatible.
tensorflow-io 0.21.0 requires tensorflow<2.7.0,>=2.6.0, but you have tensorflow 2.11.0 which is incompatible.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, but you have tensorflow-io-gcs-filesystem 0.34.0 which is incompatible.
jaxlib 0.3.14 requires flatbuffers<3.0,>=1.12, but you have flatbuffers 23.5.26 which is incompatible.


# Imports

In [2]:
import string
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_recommenders as tfrs
from collections import Counter
from typing import Dict, Text
from ast import literal_eval
from datetime import datetime
from wordcloud import WordCloud
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

# Code

In [3]:
credits = pd.read_csv('../input/the-movies-dataset/credits.csv')
keywords = pd.read_csv('../input/the-movies-dataset/keywords.csv')
movies = pd.read_csv('../input/the-movies-dataset/movies_metadata.csv').\
                     drop(['belongs_to_collection', 'homepage', 'imdb_id', 'poster_path', 'status', 'title', 'video'], axis=1).\
                     drop([19730, 29503, 35587]) # Incorrect data type

movies['id'] = movies['id'].astype('int64')

df = movies.merge(keywords, on='id').\
    merge(credits, on='id')

df['original_language'] = df['original_language'].fillna('')
df['runtime'] = df['runtime'].fillna(0)
df['tagline'] = df['tagline'].fillna('')

df.dropna(inplace=True)

In [4]:
df.head()

,adult,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,keywords,cast,crew
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",,7.7,5415.0,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,6.9,2413.0,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Still Yelling. Still Fighting. Still Ready for...,6.5,92.0,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Friends are the people who let you be yourself...,6.1,34.0,"[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Just When His World Is Back To Normal... He's ...,5.7,173.0,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45558 entries, 0 to 46627
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   adult                 45558 non-null  object 
 1   budget                45558 non-null  object 
 2   genres                45558 non-null  object 
 3   id                    45558 non-null  int64  
 4   original_language     45558 non-null  object 
 5   original_title        45558 non-null  object 
 6   overview              45558 non-null  object 
 7   popularity            45558 non-null  object 
 8   production_companies  45558 non-null  object 
 9   production_countries  45558 non-null  object 
 10  release_date          45558 non-null  object 
 11  revenue               45558 non-null  float64
 12  runtime               45558 non-null  float64
 13  spoken_languages      45558 non-null  object 
 14  tagline               45558 non-null  object 
 15  vote_average       

In [6]:
df['release_date'] = pd.to_datetime(df['release_date'])
df['budget'] = df['budget'].astype('float64')
df['popularity'] = df['popularity'].astype('float64')

<a id=2 a/>
<p style="background-color:#368f8b;font-family:Trebuchet MS;font-weight:bold;color:#eff7f6;font-size:40px;text-align:center;border-radius:100px 100px">Recommender System</p>

There are lots of methods that you can use to build recommender system. This time, we are going to explore two of them from which you can create build recommender system that can give an output of a recommended movies to the users based on different features

<a id=4 a/>
<h1 style="font-family: Trebuchet MS; font-size: 25px; color: #3a5a40; text-align: left; "><b>● Deep Learning</b></h1>

Official documentation: https://www.tensorflow.org/recommenders

Tensorflow comes with a library called TensorFlow Recommenders (TFRS) for building a recommender system. It's built on Keras and aims to have a gentle learning curve while still giving you the flexibility to build complex models.

This time, we use multi-objective approach that applies both implicit (movie watches) and explicit signals (ratings). In the end, we can predict what movies should the user watch along with the given rating corresponds to historical data

In [7]:
ratings_df = pd.read_csv('../input/the-movies-dataset/ratings_small.csv')

ratings_df['date'] = ratings_df['timestamp'].apply(lambda x: datetime.fromtimestamp(x))
ratings_df.drop('timestamp', axis=1, inplace=True)

ratings_df = ratings_df.merge(df[['id', 'original_title', 'genres', 'overview']], left_on='movieId',right_on='id', how='left')
ratings_df = ratings_df[~ratings_df['id'].isna()]
ratings_df.drop('id', axis=1, inplace=True)
ratings_df.reset_index(drop=True, inplace=True)

ratings_df.head()

,userId,movieId,rating,date,original_title,genres,overview
0,1,1371,2.5,2009-12-14 02:52:15,Rocky III,"[{'id': 18, 'name': 'Drama'}]","Now the world champion, Rocky Balboa is living..."
1,1,1405,1.0,2009-12-14 02:53:23,Greed,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",Greed is the classic 1924 silent film by Erich...
2,1,2105,4.0,2009-12-14 02:52:19,American Pie,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...","At a high-school party, four friends find that..."
3,1,2193,2.0,2009-12-14 02:53:18,My Tutor,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",High school senior Bobby Chrystal fails his Fr...
4,1,2294,2.0,2009-12-14 02:51:48,Jay and Silent Bob Strike Back,"[{'id': 35, 'name': 'Comedy'}]",When Jay and Silent Bob learn that their comic...


In [8]:
movies_df = df[['id', 'original_title']]
movies_df.rename(columns={'id':'movieId'}, inplace=True)
movies_df.head()

,movieId,original_title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


In [9]:
ratings_df['userId'] = ratings_df['userId'].astype(str)

ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_df[['userId', 'original_title', 'rating']]))
movies = tf.data.Dataset.from_tensor_slices(dict(movies_df[['original_title']]))

ratings = ratings.map(lambda x: {
    "original_title": x["original_title"],
    "userId": x["userId"],
    "rating": float(x["rating"])
})

movies = movies.map(lambda x: x["original_title"])

In [10]:
print('Total Data: {}'.format(len(ratings)))

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = ratings.take(35_000)
test = ratings.skip(35_000).take(8_188)

Total Data: 44875


In [11]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000).map(lambda x: x["userId"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

print('Unique Movies: {}'.format(len(unique_movie_titles)))
print('Unique users: {}'.format(len(unique_user_ids)))

Unique Movies: 42373
Unique users: 671


In [12]:
class MovieModel(tfrs.models.Model):

    def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
        # We take the loss weights in the constructor: this allows us to instantiate
        # several model objects with different loss weights.

        super().__init__()

        embedding_dimension = 64

        # User and movie models.
        self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential([
          tf.keras.layers.StringLookup(
            vocabulary=unique_movie_titles, mask_token=None),
          tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
        ])
        self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
          tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
          tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
        ])

        # A small model to take in user and movie embeddings and predict ratings.
        # We can make this as complicated as we want as long as we output a scalar
        # as our prediction.
        self.rating_model = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(128, activation="relu"),
            tf.keras.layers.Dense(1),
        ])

        # The tasks.
        self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()],
        )
        self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=movies.batch(128).map(self.movie_model)
            )
        )

        # The loss weights.
        self.rating_weight = rating_weight
        self.retrieval_weight = retrieval_weight

    def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        # We pick out the user features and pass them into the user model.
        user_embeddings = self.user_model(features["userId"])
        # And pick out the movie features and pass them into the movie model.
        movie_embeddings = self.movie_model(features["original_title"])

        return (
            user_embeddings,
            movie_embeddings,
            # We apply the multi-layered rating model to a concatentation of
            # user and movie embeddings.
            self.rating_model(
                tf.concat([user_embeddings, movie_embeddings], axis=1)
            ),
        )


    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        ratings = features.pop("rating")

        user_embeddings, movie_embeddings, rating_predictions = self(features)

        # We compute the loss for each task.
        rating_loss = self.rating_task(
            labels=ratings,
            predictions=rating_predictions,
        )
        retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

        # And combine them using the loss weights.
        return (self.rating_weight * rating_loss + self.retrieval_weight * retrieval_loss)

In [13]:
model = MovieModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_train = train.shuffle(100_000).batch(1_000).cache()
cached_test = test.batch(1_000).cache()

model.fit(cached_train, epochs=3)

Epoch 1/3
35/35 [==============================] - 98s 3s/step - root_mean_squared_error: 1.4235 - factorized_top_k/top_1_categorical_accuracy: 3.4286e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0065 - factorized_top_k/top_10_categorical_accuracy: 0.0157 - factorized_top_k/top_50_categorical_accuracy: 0.0894 - factorized_top_k/top_100_categorical_accuracy: 0.1547 - loss: 6814.1192 - regularization_loss: 0.0000e+00 - total_loss: 6814.1192
Epoch 2/3
35/35 [==============================] - 93s 3s/step - root_mean_squared_error: 1.0278 - factorized_top_k/top_1_categorical_accuracy: 0.0011 - factorized_top_k/top_5_categorical_accuracy: 0.0168 - factorized_top_k/top_10_categorical_accuracy: 0.0390 - factorized_top_k/top_50_categorical_accuracy: 0.1807 - factorized_top_k/top_100_categorical_accuracy: 0.2971 - loss: 6450.3738 - regularization_loss: 0.0000e+00 - total_loss: 6450.3738
Epoch 3/3
35/35 [==============================] - 92s 3s/step - root_mean_squared_error: 0.9818 - fac

In [14]:
metrics = model.evaluate(cached_test, return_dict=True)

print(f"\nRetrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}")

9/9 [==============================] - 17s 2s/step - root_mean_squared_error: 1.0872 - factorized_top_k/top_1_categorical_accuracy: 6.1065e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0050 - factorized_top_k/top_10_categorical_accuracy: 0.0094 - factorized_top_k/top_50_categorical_accuracy: 0.0445 - factorized_top_k/top_100_categorical_accuracy: 0.0798 - loss: 5723.5685 - regularization_loss: 0.0000e+00 - total_loss: 5723.5685

Retrieval top-100 accuracy: 0.080
Ranking RMSE: 1.087


In [15]:
def predict_movie(user, top_n=3):
    # Create a model that takes in raw query features, and
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    # recommends movies out of the entire movies dataset.
    index.index_from_dataset(
      tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
    )

    # Get recommendations.
    _, titles = index(tf.constant([str(user)]))
    
    print('Top {} recommendations for user {}:\n'.format(top_n, user))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print('{}. {}'.format(i+1, title.decode("utf-8")))

def predict_rating(user, movie):
    trained_movie_embeddings, trained_user_embeddings, predicted_rating = model({
          "userId": np.array([str(user)]),
          "original_title": np.array([movie])
      })
    print("Predicted rating for {}: {}".format(movie, predicted_rating.numpy()[0][0]))

# Prediction

### UserId: 123

In [16]:
predict_movie(123, 5)

Top 5 recommendations for user 123:

1. The Greatest Story Ever Told
2. Un long dimanche de fiançailles
3. El otro lado de la cama
4. Dog Day Afternoon
5. Un éléphant ça trompe énormément


In [17]:
predict_rating(123,'Minions')

Predicted rating for Minions: 2.9657320976257324


So, le't examine **User 123** from historical data

In [18]:
ratings_df[ratings_df['userId'] == '123']

,userId,movieId,rating,date,original_title,genres,overview
8360,123,233,4.0,2001-07-01 20:57:06,The Wanderers,"[{'id': 18, 'name': 'Drama'}]",The streets of the Bronx are owned by 60’s you...
8361,123,288,5.0,2001-07-01 19:32:47,High Noon,"[{'id': 37, 'name': 'Western'}]",High Noon is about a recently freed leader of ...
8362,123,407,5.0,2001-07-01 20:57:57,Kurz und schmerzlos,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",Three friends get caught in a life of major cr...
8363,123,968,3.0,2001-07-01 20:59:01,Dog Day Afternoon,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",A man robs a bank to pay for his lover's opera...
8364,123,1968,4.0,2001-07-01 19:30:36,Fools Rush In,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",Alex Whitman (Matthew Perry) is a designer fro...
8365,123,1976,4.0,2001-07-01 19:31:51,Jezebel,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","In 1850s Louisiana, the willfulness of a tempe..."
8366,123,2003,4.0,2001-07-01 19:31:51,Anatomie de l'enfer,"[{'id': 18, 'name': 'Drama'}]",A man rescues a woman from a suicide attempt i...
8367,123,2428,5.0,2001-07-01 20:57:06,The Greatest Story Ever Told,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",All-star epic retelling of Christ's life.
8368,123,2502,5.0,2001-07-01 20:59:01,The Bourne Supremacy,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",When a CIA operation to purchase classified Ru...
8369,123,2699,2.0,2001-07-01 20:55:42,Titanic,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",A story of the romances of two couples upon th...


In [19]:
# Get meta data for predicted movie
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get recommendations.
_, titles = index(tf.constant(['123']))
pred_movies = pd.DataFrame({'original_title': [i.decode('utf-8') for i in titles[0,:5].numpy()]})

pred_df = pred_movies.merge(ratings_df[['original_title', 'genres', 'overview']], on='original_title', how='left')
pred_df = pred_df[~pred_df['original_title'].duplicated()]
pred_df.reset_index(drop=True, inplace=True)
pred_df.index = np.arange(1, len(pred_df)+1)

pred_df

,original_title,genres,overview
1,The Greatest Story Ever Told,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",All-star epic retelling of Christ's life.
2,Un long dimanche de fiançailles,"[{'id': 18, 'name': 'Drama'}]","In 1919, Mathilde was 19 years old. Two years ..."
3,El otro lado de la cama,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...","When Paula leaves her mate Pedro, he misses he..."
4,Dog Day Afternoon,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",A man robs a bank to pay for his lover's opera...
5,Un éléphant ça trompe énormément,"[{'id': 35, 'name': 'Comedy'}]",Etienne falls in love with a sexy woman he mee...


At a glance, we can see if **User 123** love watching Drama movies most of the time. He/She also gives a good rating for that genre. In our recommendation, We give 5 more Drama movies that we expect him/her to love the movies in a similar way with the previous watched movies.

In our dataset, we don't see any Animation movies that have been watched by **User 123**. So, it's not a surprise if the estimated rating for Minions is quite low

### User: 99

In [20]:
predict_movie(99, 5)

Top 5 recommendations for user 99:

1. Bringing Up Baby
2. Birdman of Alcatraz
3. Double Dragon
4. Blind Date
5. Blind Date


In [21]:
predict_rating(99,'Toy Story')

Predicted rating for Toy Story: 2.8420114517211914


**Lets examine User:99 from historical data**

In [22]:
ratings_df[ratings_df['userId'] == '99']

,userId,movieId,rating,date,original_title,genres,overview
6589,99,2,2.0,2000-02-19 22:39:18,Ariel,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",Taisto Kasurinen is a Finnish coal miner whose...
6590,99,17,3.0,1999-09-29 06:36:39,The Dark,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",Adèle and her daughter Sarah are traveling on ...
6591,99,28,3.0,1999-09-29 06:36:39,Apocalypse Now,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...","At the height of the Vietnam war, Captain Benj..."
6592,99,105,3.0,1999-09-29 06:43:27,Back to the Future,"[{'id': 12, 'name': 'Adventure'}, {'id': 35, '...",Eighties teenager Marty McFly is accidentally ...
6593,99,110,3.0,2000-01-10 15:51:40,Trois couleurs : Rouge,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...",Red This is the third film from the trilogy by...
...,...,...,...,...,...,...,...
6689,99,4641,5.0,2002-03-02 02:57:04,Read It and Weep,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",A young girl turns into an A-List celebrity ov...
6690,99,4973,5.0,2002-03-02 02:56:51,Sous le Sable,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","When her husband goes missing at the beach, a ..."
6691,99,4993,5.0,2003-02-09 10:35:19,5 Card Stud,"[{'id': 28, 'name': 'Action'}, {'id': 37, 'nam...",The players in an ongoing poker game are being...
6692,99,4995,5.0,2002-03-02 02:54:22,Boogie Nights,"[{'id': 18, 'name': 'Drama'}]","Set in 1977, back when sex was safe, pleasure ..."


In [23]:
# Get meta data for predicted movie
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get recommendations.
_, titles = index(tf.constant(['99']))
pred_movies = pd.DataFrame({'original_title': [i.decode('utf-8') for i in titles[0,:5].numpy()]})

pred_df = pred_movies.merge(ratings_df[['original_title', 'genres', 'overview']], on='original_title', how='left')
pred_df = pred_df[~pred_df['original_title'].duplicated()]
pred_df.reset_index(drop=True, inplace=True)
pred_df.index = np.arange(1, len(pred_df)+1)

pred_df

,original_title,genres,overview
1,Bringing Up Baby,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",David Huxley is waiting to get a bone he needs...
2,Birdman of Alcatraz,"[{'id': 18, 'name': 'Drama'}]","After killing a prison guard, convict Robert S..."
3,Double Dragon,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Set 15 years in the future in post-earthquake ...
4,Blind Date,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",When bachelor Walter Davis (Bruce Willis) is s...


At a glance, we can see if **User 99** love watching Comedy, Action genre movies most of the time. He/She also gives a good rating for that genre. In our recommendation, We give 5 more Drama movies that we expect him/her to love the movies in a similar way with the previous watched movies.

In our dataset, we don't see any Animation movies that have been watched by **User 99**. So, it's not a surprise if the estimated rating for Toy Story is quite low

<h1 style="font-family: Trebuchet MS; font-size: 60px; color: #1b4332; text-align: center;"><b>THE END</b></h1>